In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import time
from tqdm import tqdm
import os
import json

In [2]:
df = pd.read_parquet(r"Z:\BSDS3002_GP_GIT\BSDS3002\Part2 - Results\April_latest_artists_with_relations.parquet")

In [3]:
unique_values = df['Genres'].explode().unique()

In [5]:
checkpoint_file = 'id_gen_checkpoint.json'
if os.path.exists(checkpoint_file):
    with open(checkpoint_file, 'r') as f:
        genre_dict = json.load(f)
else:
    genre_dict = {}

progress_bar = tqdm(total=len(unique_values))

for val in unique_values:
    if not val:
        continue
    if val in genre_dict:
        progress_bar.update(1)
        continue
    # print(genre.replace(' ', '-').lower())
    genre = val.replace(' ', '-').lower()
    url = f'https://www.chosic.com/genre-chart/{genre}/'
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
    }

    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        print(f"Error: {val} not found")
        continue
    soup = BeautifulSoup(response.content, 'html.parser')

    parent_genre_element = soup.find('div', attrs={'class':'parent-genre'})

    if parent_genre_element:
        parent_genre_href = parent_genre_element.find('a').get('href')
        parent_genre = parent_genre_href.split('/')[-2]
        if not parent_genre == genre:
            genre_dict[val] = parent_genre
        else:
            genre_dict[val] = 'is_parent'
    else:
        genre_dict[val] = None
    time.sleep(0.5)

    # Save the current state to the checkpoint file
    with open(checkpoint_file, 'w') as f:
        json.dump(genre_dict, f)

    # Update the progress bar with each completed item
    progress_bar.update(1)

# Close the progress bar when finished
progress_bar.close()


Error: bosstown sound not found


 17%|█▋        | 366/2199 [10:22<51:57,  1.70s/it]


Error: urbano latino not found


Error: crank wave not found


Error: pixel not found


KeyboardInterrupt: 

In [ ]:
genre = 'metal'

url = f'https://www.chosic.com/genre-chart/{genre}/'
headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'
}
response = requests.get(url, headers=headers)
response.raise_for_status()
soup = BeautifulSoup(response.content, 'html.parser')

parent_genre_element = soup.find('div', attrs={'class':'parent-genre'})

if parent_genre_element:
    parent_genre_href = parent_genre_element.find('a').get('href')
    parent_genre = parent_genre_href.split('/')[-2]
    if not parent_genre == genre:
        print(f'Parent genre of {genre}: {parent_genre}')
    else:
        print('same')
else:
    print(f'{genre} has no parent genre')